In [1]:
import requests
import datetime
import base64
import shutil
import json
import time
import os.path
from urllib.parse import urlencode

In [2]:
from config import client_id, client_secret

In [3]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_is_expired = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        
        if client_id == None or client_secret == None:
            raise Exception("Requires client_id and client_secret")
            
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Unable to authenticate client.")
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_is_expired = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_playlist_items(self, lookup_id, query, limit, offset, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}/tracks"
        headers = self.get_resource_header()
        data = urlencode({"fields": query, "limit":limit, "offset":offset })
        lookup_url = f"{endpoint}?{data}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, lookup_id):
        return self.get_resource(lookup_id, resource_type='albums')
    
    def get_track(self, lookup_id):
        return self.get_resource(lookup_id, resource_type='tracks')
    
    def get_artist(self, lookup_id):
        return self.get_resource(lookup_id, resource_type='artists')
    
    def get_artist_albums(self, lookup_id, version='v1'):
        endpoint = f"https://api.spotify.com/{version}/artists/{lookup_id}/albums"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
        
    def search(self, query, search_type='artist' ): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q": query, "type": search_type.lower()})
        lookup_url = f"{endpoint}?{data}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json() 

In [4]:
spotify = SpotifyAPI(client_id, client_secret)

In [5]:
r = spotify.search('Most%Popular%Songs', search_type = 'playlist')

In [ ]:
r = spotify.get_resource('4o8NBsWreC3OnKePUYw0dg', 'playlists')
print(json.dumps(r, indent=4))

In [11]:
with open('data.txt') as json_file:
    data = json.load(json_file)



for item in data.keys():
    print(item, end= ' ')
    downloaded = False
    
    while not downloaded:
        try:
            image_url = data[item]['album_url']
            album_id = data[item]['album_id'].replace('/','')
            filename = ".\\images\\" + album_id + '.jpg'
        except:
            break
        
        if os.path.isfile(filename):
            print('File already downloaded.')
            break
        try:
            r = requests.get(image_url, timeout=200, stream = True)
            time.sleep(1)
        except:
            continue

        if r.status_code == 200:
            r.raw.decode_content = True

            with open(filename,'wb') as f:
                shutil.copyfileobj(r.raw, f)
            #print('Image sucessfully Downloaded: ',filename)
            downloaded = True
            data[item]['filepath'] = filename
        else:
            print('Image Couldn\'t be retrieved')

0 File already downloaded.
1 File already downloaded.
2 File already downloaded.
3 File already downloaded.
4 File already downloaded.
5 File already downloaded.
6 File already downloaded.
7 File already downloaded.
8 File already downloaded.
9 File already downloaded.
10 File already downloaded.
11 File already downloaded.
12 File already downloaded.
13 File already downloaded.
14 File already downloaded.
15 File already downloaded.
16 File already downloaded.
17 File already downloaded.
18 File already downloaded.
19 File already downloaded.
20 File already downloaded.
21 File already downloaded.
22 File already downloaded.
23 File already downloaded.
24 File already downloaded.
25 File already downloaded.
26 File already downloaded.
27 File already downloaded.
28 File already downloaded.
29 File already downloaded.
30 File already downloaded.
31 File already downloaded.
32 File already downloaded.
33 File already downloaded.
34 File already downloaded.
35 File already downloaded.
36

767 File already downloaded.
768 File already downloaded.
769 File already downloaded.
770 File already downloaded.
771 File already downloaded.
772 File already downloaded.
773 File already downloaded.
774 File already downloaded.
775 File already downloaded.
776 File already downloaded.
777 File already downloaded.
778 File already downloaded.
779 File already downloaded.
780 File already downloaded.
781 File already downloaded.
782 File already downloaded.
783 File already downloaded.
784 File already downloaded.
785 File already downloaded.
786 File already downloaded.
787 File already downloaded.
788 File already downloaded.
789 File already downloaded.
790 File already downloaded.
791 File already downloaded.
792 File already downloaded.
793 File already downloaded.
794 File already downloaded.
795 File already downloaded.
796 File already downloaded.
797 File already downloaded.
798 File already downloaded.
799 File already downloaded.
800 File already downloaded.
801 File alrea

1578 File already downloaded.
1579 File already downloaded.
1580 File already downloaded.
1581 File already downloaded.
1582 File already downloaded.
1583 File already downloaded.
1584 File already downloaded.
1585 File already downloaded.
1586 File already downloaded.
1587 File already downloaded.
1588 File already downloaded.
1589 File already downloaded.
1590 File already downloaded.
1591 File already downloaded.
1592 File already downloaded.
1593 File already downloaded.
1594 File already downloaded.
1595 File already downloaded.
1596 File already downloaded.
1597 File already downloaded.
1598 File already downloaded.
1599 File already downloaded.
1600 File already downloaded.
1601 File already downloaded.
1602 File already downloaded.
1603 File already downloaded.
1604 File already downloaded.
1605 File already downloaded.
1606 File already downloaded.
1607 File already downloaded.
1608 File already downloaded.
1609 File already downloaded.
1610 File already downloaded.
1611 File 

2215 2216 2217 File already downloaded.
2218 2219 File already downloaded.
2220 2221 2222 2223 2224 File already downloaded.
2225 File already downloaded.
2226 File already downloaded.
2227 File already downloaded.
2228 2229 2230 2231 File already downloaded.
2232 File already downloaded.
2233 File already downloaded.
2234 File already downloaded.
2235 File already downloaded.
2236 2237 File already downloaded.
2238 2239 2240 2241 File already downloaded.
2242 File already downloaded.
2243 2244 2245 2246 File already downloaded.
2247 2248 File already downloaded.
2249 2250 File already downloaded.
2251 2252 2253 File already downloaded.
2254 2255 2256 2257 File already downloaded.
2258 File already downloaded.
2259 File already downloaded.
2260 File already downloaded.
2261 2262 2263 2264 2265 2266 File already downloaded.
2267 File already downloaded.
2268 File already downloaded.
2269 File already downloaded.
2270 2271 2272 2273 2274 2275 2276 2277 2278 2279 2280 2281 2282 2283 File 

In [13]:
with open('data_extra_v2.txt', 'w') as file:
    json.dump(data, file)

In [22]:
with open('data_extra_v2.txt') as json_file:
    data = json.load(json_file)

for item in data.keys():
    for i in data[item].keys():
        print(i)
        

artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
ar

album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
alb

genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album

album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
a

artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
ar

genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album

filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_

album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url


filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
filepath
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_id
artist_name
album_id
album_name
album_popularity
genres
album_url
artist_